In [ ]:
import os
path = os.getcwd()
path

In [ ]:
import json
import os

path = path

l = []
l1 = []
#importing data from the provided datasets
def get_text(path_to_json):
    with open(path_to_json) as json_file:
        data = json.load(json_file)
        train = list(data.values())
        l = []
        l1 = []
        for item in range(len(train)):
            l.append(train[item]['text'])
            l1.append(train[item]['label'])
        
    return l, l1  
    
#label-0 : Not a misinformtion
#label-1 : It is a misinformation

train_doc, train_label = get_text(os.path.join(path, 'train.json'))
print(len(train_doc))
dev_doc, dev_label = get_text(os.path.join(os.path.join(path, 'dev.json')))
print(len(dev_doc))


def get_test_data(path_to_json):
    with open(path_to_json) as json_file:
        data = json.load(json_file)
        train = list(data.values())
        l = []
        #l1 = []
        for item in range(len(train)):
            l.append(train[item]['text'])
            #l1.append(train[item]['label'])
        
    return l 
test_doc = get_test_data(os.path.join(os.path.join(path, 'test-unlabelled.json')))
print(len(test_doc))




In [ ]:
#get the external data set
import json

with open('external_data.json') as json_file:
    external_data = json.load(json_file)

In [ ]:
X_train = train_doc + external_data
y_train = train_label + [0]*1275 + [1]*16

In [ ]:
print(len(X_train))
print(len(y_train))

In [ ]:
#Data Preprocessing

import gensim

def sent_to_words(sentences):
    x1=[]
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
new_train_X1 = list(sent_to_words(X_train))
dev_X1 = list(sent_to_words(dev_doc))
test_X1 = list(sent_to_words(test_doc))

In [ ]:
#Stopwords removal
import nltk
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))

l = []
def stopwords_remove(doc):
    l = []
    for i in range(len(doc)):
        l.append([word for word in doc[i] if not word in stopwords])
        l[i] = (' '.join([w for w in l[i]]))
    
    return l

In [ ]:
train_1 = stopwords_remove(new_train_X1)
dev_1 = stopwords_remove(dev_X1)
test_1 = stopwords_remove(test_X1)

In [ ]:
# POS Tagging
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


# 1. Init Lemmatizer
lemmatizer = WordNetLemmatizer()

#Lemmatization after POS Tagging
def lempos(doc):
    X = []
    for i in range(len(doc)):
        X.append([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(doc[i])])
        X[i] = (' '.join([w for w in X[i]]))
    return X





In [ ]:
train_set = lempos(train_1)
dev_set = lempos(dev_1)
test_set = lempos(test_1)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score

In [ ]:
# Count Vectorizer

count_vect = CountVectorizer()


train_count = count_vect.fit_transform(train_set)
dev_count = count_vect.transform(dev_set)
test_count = count_vect.transform(test_set)

tfidf_trans = TfidfTransformer()
tc = tfidf_trans.fit_transform(train_count)
dc = tfidf_trans.transform(dev_count)
testc = tfidf_trans.transform(test_count)





In [ ]:
# Hashing Vectorizer



hash_vect = HashingVectorizer()

train_count1 = hash_vect.fit_transform(train_set)
dev_count1 = hash_vect.transform(dev_set)
test_count1 = hash_vect.transform(test_set)



In [ ]:
# TF-IDF Vectorizer

tf_vect = TfidfVectorizer(
                         )

train_count2 = tf_vect.fit_transform(train_set)
dev_count2 = tf_vect.transform(dev_set)
test_count2 = tf_vect.transform(test_set)



In [ ]:
# K Nearest Neighbors

#knn = KNeighborsClassifier()
knn = KNeighborsClassifier(n_neighbors=5)
mod1 = knn.fit(train_count2, y_train)
pred1 = mod1.predict(dev_count2)
pred1_f = mod1.predict(test_count2)

print(f1_score(dev_label, pred1, average='macro'))
print(accuracy_score(dev_label, pred1))

In [ ]:
#Error Analysis
indices1 = [i for i in range(len(dev_label)) if dev_label[i] != pred1[i]]
print(len(indices1))
print(indices1)



In [ ]:
# Support Vector Machine

from sklearn import svm


gammas = [0.1, 1, 10, 100]
cs = [0.1, 1, 10, 100, 1000]
degrees = [0, 1, 2, 3, 4, 5, 6]
SVM = svm.SVC(C=10)
#SVM = svm.SVC(kernel='poly', gamma=0.1, C=10, degree=2)
mod2 = SVM.fit(train_count2, y_train)
pred2 = mod2.predict(dev_count2)
pred2_f = mod2.predict(test_count2)

print(f1_score(dev_label, pred2, average='macro'))
print(accuracy_score(dev_label, pred2))

In [ ]:
#Error Analysis
indices5 = [i for i in range(len(dev_label)) if dev_label[i] != pred2[i]]
print(len(indices5))
print(indices5)

In [ ]:
# Naive Bayes 

from sklearn import naive_bayes

MNB = naive_bayes.MultinomialNB(alpha=0.001, fit_prior=False)
BNB = naive_bayes.BernoulliNB(alpha=0.001)
GNB = naive_bayes.GaussianNB()
mod3 = MNB.fit(tc, y_train)
pred3 = mod3.predict(dc)
pred3_f = mod3.predict(testc)

print(f1_score(dev_label, pred3, average='macro'))
print(accuracy_score(dev_label, pred3))

In [ ]:
indices4 = [i for i in range(len(dev_label)) if dev_label[i] != pred3[i]]
print(len(indices4))
print(indices4)

In [ ]:
# Logistic Regression

from sklearn.linear_model import LogisticRegression

C = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]

log_reg = LogisticRegression(C=1000, n_jobs=-1)
mod4 = log_reg.fit(train_count2, y_train)
pred4 = mod4.predict(dev_count2)
pred4_f = mod4.predict(test_count2)

print(f1_score(dev_label, pred4, average='macro'))
print(accuracy_score(dev_label, pred4))

In [ ]:
#Error Analysis
indices = [i for i in range(len(dev_label)) if dev_label[i] != pred4[i]]
print(len(indices))
print(indices)

In [ ]:
# Random Forest Classifier

from sklearn.ensemble import RandomForestClassifier
#RFC = RandomForestClassifier()
n_estimators = [1, 2, 4, 8, 16, 32, 64, 100, 200, 300]
maxfeat = ['sqrt', 'log2', None]
#for nest in n_estimators:
#    for mf in maxfeat:
RFC = RandomForestClassifier(n_estimators=200, 
                               max_features = 'sqrt', 
                               n_jobs=-1, verbose = 1, random_state=1)
mod5 = RFC.fit(train_count2, y_train)
pred5 = mod5.predict(dev_count2)
pred5_f = mod5.predict(test_count2)
print(f1_score(dev_label, pred5, average='macro'))
print(accuracy_score(dev_label, pred5))
       # print("The parameters for this model are: "+ str(nest), str(mf))

In [ ]:
indices6 = [i for i in range(len(dev_label)) if dev_label[i] != pred5[i]]
print(len(indices6))
print(indices6)

In [ ]:
from sklearn.ensemble import VotingClassifier

estimators = [ #IsolationForest(n_jobs=-1)),
    ('rfc', RandomForestClassifier(n_estimators=50, 
                               max_features = 'sqrt',
                               n_jobs=-1, verbose = 1)),
    ('lg', LogisticRegression(C=1000))
   # ('knn', KNeighborsClassifier(n_neighbors=5)),
    #('svm', svm.SVC(C=1))
    #('nbm', naive_bayes.MultinomialNB(alpha=0.001, fit_prior=False))
    ]
clf1 = VotingClassifier(estimators=estimators, n_jobs=-1)

mod8 = clf1.fit(train_count2, y_train)
pred8 = mod8.predict(dev_count2)
pred8_f = mod8.predict(test_count2)

print(f1_score(dev_label, pred8, average='macro'))
print(accuracy_score(dev_label, pred8))



In [ ]:
indices9 = [i for i in range(len(dev_label)) if dev_label[i] != pred8[i]]
print(len(indices9))
print(indices9)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
estimators = [#('xgb', XGBClassifier()), 
   # ('etc', ExtraTreesClassifier()),
    ('rfc', RandomForestClassifier(n_estimators=50, 
                               max_features = 'sqrt',
                               n_jobs=-1, verbose = 2, random_state=42)),
    ('lg', LogisticRegression(C=10000)),
    ('svm', svm.SVC(C=1))
     ]
clf = StackingClassifier(estimators=estimators, final_estimator = svm.SVC(C=1), cv=10,
                         passthrough=True  #, stack_method='predict_proba'
                     )

mod5 = clf.fit(train_count2, y_train)
pred5 = mod5.predict(dev_count2)
pred5_f = mod5.predict(test_count2)

print(f1_score(dev_label, pred5, average='macro'))
print(accuracy_score(dev_label, pred5))

In [ ]:
indices6 = [i for i in range(len(dev_label)) if dev_label[i] != pred5[i]]
print(len(indices6))
print(indices6)



In [ ]:
#function to create format for the test submission Files
def str_json_format(results):
    strings = "{"
    for i, j in enumerate(list(results)):
        j = str(j)
        i = str(i)
        strings += "\"test-" + i + "\"" + ":{\"label\":" + j + "},"
    strings = strings[:-1] + "}"
    
    return strings



In [ ]:
from itertools import chain

In [ ]:
X3 = list(chain(*dev_set))
print(len(X3))

In [ ]:
print(Counter(X3).most_common(100))